In [121]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
import requests
from bs4 import BeautifulSoup
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.common.action_chains import ActionChains

import time


In [122]:
chrome_driver_path = 'chromedriver'

In [123]:
str_username = 'Enter Your Username Here'
str_password = 'Enter Your Password Here'

In [124]:
# Using Chrome to access web
driver = webdriver.Chrome(executable_path=chrome_driver_path)
# Open the website
driver.get('https://www.iimjobs.com/jobfeed')

/var/folders/07/9fnf9n1n6d377009rz_968kc0000gp/T/ipykernel_67333/1008131508.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=chrome_driver_path)


### Entering username and password

In [125]:
# To catch <input type="text" id="large-email" />
username = driver.find_element(By.ID, "large-email")
# To catch <input type="text" id="large-password" />
password = driver.find_element(By.ID, "large-password")

username.send_keys(str_username)
password.send_keys(str_password)

driver.find_element(By.ID, "open-modal").click()

### Applying filters

In [126]:
#yoe filter
driver.find_element(By.XPATH, '//*[@id="searchform"]/div[1]/div[2]/a/span[1]').click()
driver.find_element(By.LINK_TEXT, '4-6 yrs').click()
#location filter
driver.find_element(By.XPATH, '//*[@id="searchform"]/div[1]/div[3]/div/div[1]/a/span[1]').click()
driver.find_element(By.XPATH, '//*[@id="dvloc"]/div[8]/label').click() ## xpath for bangalore used
driver.find_element(By.XPATH, '//*[@id="searchform"]/div[1]/div[3]/div/div[1]/a/span[1]').click()
#click on filter button
driver.find_element(By.XPATH, '//*[@id="searchform"]/div[1]/div[5]/input').click()

In [127]:
## getting url after applying filters
filter_url = driver.current_url


### Scraping all vacancies

In [128]:
# parser-lxml = Change html to Python friendly format
# Obtain page's information
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [129]:
# find profile section to parse required information
# div_profile_right = soup.find_all('div', {'id':'listingPanel', 'class':'col-md-9 pdlr0'})
# div_profile_right = soup.find_all('div', {'class':'listing'})
div_profile_right = soup.find_all('a', {'class':'mrmob5 hidden-xs'})

### Creating list of jobs

In [130]:
job_list = []
job_link_list = []
for job in div_profile_right:
    job_list.append(job.contents[0].strip())
    job_link_list.append(job['href'])

### Applying for each of the jobs

In [131]:
job_list_status = []

In [132]:
%%time
for (ix, job_link) in enumerate(job_link_list):
    driver.get(job_link)
    apply_button_xpath = '//*[@id="job-apply-single"]'
    submit_button_xpath = '//*[@id="applyJob"]'
#     apply_button_xpath = '//*[@id="reportjob"]'
#     apply_button_xpath = '//*[@id="page-content-wrapper"]/div[9]/div[4]/div[3]/b'
    delay = 5 # seconds
    current_time = time.time()
    try:
        myElem = WebDriverWait(driver, delay)\
                .until(EC.presence_of_element_located((By.XPATH, apply_button_xpath)))
        driver.find_element(By.XPATH, apply_button_xpath).click()  
        myElem = WebDriverWait(driver, delay)\
                .until(EC.presence_of_element_located((By.XPATH, submit_button_xpath)))
        driver.find_element(By.XPATH, submit_button_xpath).click()
        job_list_status.append([job_list[ix], job_link, 'Applied', current_time])
    except:
        job_list_status.append([job_list[ix], job_link, 'Not Applied', current_time])
    
        

In [134]:
job_status_df = pd.DataFrame(job_list_status, columns=['Job', 'Link', 'Status', 'Time Stamp'])

In [ ]:
with pd.ExcelWriter('output.xlsx', mode='a') as writer:  
    df.to_excel(writer, sheet_name='Sheet_name_3')

In [135]:
job_status_df.groupby('Status').count()

,Job,Link
Status,,
Applied,26,26
Not Applied,29,29


In [133]:
job_list_status

[['Quizizz - Product Manager (3-12 yrs)',
  'https://www.iimjobs.com/j/quizizz-product-manager-3-12-yrs-1172784.html?ref=jf',
  'Not Applied'],
 ['Senior Product Manager - Consumer Growth - FinTech - IIM/ISB (5-10 yrs)',
  'https://www.iimjobs.com/j/senior-product-manager-consumer-growth-fintech-iim-isb-5-10-yrs-1171603.html?ref=jf',
  'Not Applied'],
 ['take2.ai - Product Analyst (3-10 yrs)',
  'https://www.iimjobs.com/j/take2ai-product-analyst-3-10-yrs-1170666.html?ref=jf',
  'Not Applied'],
 ['Product Analyst - Online/Internet (3-5 yrs)',
  'https://www.iimjobs.com/j/product-analyst-online-internet-3-5-yrs-1170665.html?ref=jf',
  'Applied'],
 ['MergerWare.com - Product Manager (3-12 yrs)',
  'https://www.iimjobs.com/j/mergerwarecom-product-manager-3-12-yrs-1170090.html?ref=jf',
  'Not Applied'],
 ['Product Manager/Senior Product Manager - SaaS (3-7 yrs)',
  'https://www.iimjobs.com/j/product-manager-senior-product-manager-saas-3-7-yrs-1169955.html?ref=jf',
  'Applied'],
 ['Matrimony